In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
data_X = mnist.train.images

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
#Data parameters
image_size = 28*28
n_class = 10

#Training parameters
n_cluster = 25
total_epochs = 50

X = tf.placeholder(tf.float32, shape=[None, image_size], name='InputImage')
Y = tf.placeholder(tf.float32, shape=[None, n_class], name="Label")
kmeans = KMeans(inputs=X, num_clusters=n_cluster, distance_metric='squared_euclidean', use_mini_batch=True)
#all_scores: A matrix (or list of matrices) of dimensions (num_input, num_clusters) where the value is the distance of an input vector and a cluster center. 
#cluster_idx: A vector (or list of vectors). Each element in the vector corresponds to an input row in 'inp' and specifies the cluster id corresponding to the input. 
#scores: Similar to cluster_idx but specifies the distance to the assigned cluster instead. 
#cluster_centers_initialized: scalar indicating whether clusters have been initialized. 
#init_op: an op to initialize the clusters. 
#training_op: an op that runs an iteration of training.
all_scores, cluster_idx, scores, cluster_centers_initialized, init_op, train_op = kmeans.training_graph()
cluster_idx = cluster_idx[0]

init = tf.global_variables_initializer()

In [4]:
with tf.Session() as sess:
    sess.run(init)
    #init kmeans graph
    sess.run(init_op, feed_dict={X:data_X})
    for i in range(total_epochs):
        _, idx = sess.run([train_op, cluster_idx], feed_dict={X: data_X})
    #idx: shape(55000,) where its values correspond to the belonging cluster of index.
    counts = np.zeros(shape=(n_cluster, n_class))
    for i in range(len(idx)):
        counts[idx[i]] += mnist.train.labels[i]
        
    #assign the label with the largest possibility of appearance for every cluster.
    labels_map = [np.argmax(c) for c in counts]
    labels_map = tf.convert_to_tensor(labels_map)

    #lookup labels for each cluster using labels_map
    cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)
    #compute accuracy
    correct_prediction = tf.equal(cluster_label, tf.cast(tf.argmax(Y, 1), tf.int32))
    accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    #Test Model
    test_x, test_y = mnist.test.images, mnist.test.labels
    print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

Test Accuracy: 0.6963
